In [2]:
import requests
import json
from dotenv import load_dotenv
import os
import pandas as pd
load_dotenv()
import src.Funciones as fun
from functools import reduce
import operator
from pandas import json_normalize
import numpy as np
from geopy.geocoders import Nominatim
import folium
from folium import Choropleth, Circle, Marker, Icon, Map

def planazos(ciudad_destino):
    geolocator = Nominatim(user_agent="edu")
    location = geolocator.geocode(ciudad_destino)
    lat = location.latitude
    lon = location.longitude

    url_query = 'https://api.foursquare.com/v2/venues/explore'

    queries_HS = ["Historic Site"]
    queries_museum = ["Museum"]
    queries_bar = ["Nightclub"]
    queries_food = ["Food"]
    queries_hotel = ["Hotel"]

    tok1= os.getenv("Client_Id")
    tok2= os.getenv("Client_Secret")

    HS = fun.get_data(lat, lon,url_query,*queries_HS)
    museum = fun.get_data(lat, lon,url_query,*queries_museum)
    nightclub = fun.get_data(lat, lon,url_query,*queries_bar)
    food = fun.get_data(lat, lon,url_query,*queries_food)
    hoteles = fun.get_data(lat, lon,url_query,*queries_hotel)

    HS = HS.get("Historic Site").get("response").get("groups")[0].get("items")
    museum = museum.get("Museum").get("response").get("groups")[0].get("items")
    nightclub = nightclub.get("Nightclub").get("response").get("groups")[0].get("items")
    food = food.get("Food").get("response").get("groups")[0].get("items")
    hoteles = hoteles.get("Hotel").get("response").get("groups")[0].get("items")

    df_HS = fun.crear_dataframe(HS)
    df_museum = fun.crear_dataframe(museum)
    df_nightclub = fun.crear_dataframe(nightclub)
    df_food = fun.crear_dataframe(food)
    df_hoteles = fun.crear_dataframe(hoteles)

    df_HS = df_HS[:10]
    df_HS = (df_HS.assign(categoria = "historic site"))

    df_hoteles = df_hoteles[:10]
    df_hoteles = (df_hoteles.assign(categoria = "hotels"))

    df_food = df_food[:10]
    df_food = (df_food.assign(categoria = "food"))

    df_museum = df_museum[:10]
    df_museum = (df_museum.assign(categoria = "museum"))

    df_nightclub = df_nightclub[:10]
    df_nightclub = (df_nightclub.assign(categoria = "nightclub"))

    df = pd.concat([df_food,df_hoteles,df_HS,df_museum,df_nightclub],ignore_index=True)

    direcc = []
    for i, row in df.iterrows():
        lat = row["latitud"]
        lon = row["longitud"]
        location = geolocator.reverse(f'{lat}'"," f'{lon}')
        calle = location.address
        direcc.append(calle)

    df["Address"] = direcc
    return df

In [4]:
planazos("madrid")

,name,latitud,longitud,categoria,Address
0,La Pulpería de Victoria,40.416506,-3.701709,food,"2, Calle de la Victoria, Sol, Centro, Madrid, ..."
1,Rosi La Loca,40.415821,-3.702955,food,"3, Calle de Cádiz, Sol, Centro, Madrid, Área m..."
2,Ferpal,40.417046,-3.705742,food,"Edificio Arenal, Calle Arenal, Sol, Centro, Ma..."
3,TAKOS,40.418938,-3.703748,food,"Takos Al Pastor, 2, Calle Abada, Sol, Centro, ..."
4,InClan Brutal Bar,40.415023,-3.701864,food,"Inclán Brutal Bar, 4, Calle de Álvarez Gato, B..."
5,Taberna Malaspina,40.415767,-3.702644,food,"Cuevas El Secreto, 2, Calle de Barcelona, Sol,..."
6,Bacoa,40.416698,-3.701682,food,"Bacoa, 3, Carrera de San Jerónimo, Sol, Centro..."
7,Casa Labra,40.417150,-3.704514,food,"El Corte Inglés, 10, Calle de Tetuán, Sol, Cen..."
8,Taberna del Chato,40.414845,-3.702683,food,"Taberna del Chato, 35, Calle de la Cruz, Sol, ..."
9,Museo del Jamón,40.416343,-3.705682,food,"Museo del Jamón, Calle de San Cristóbal, Sol, ..."
